# Suffix arrays

#### Useful links:
* Wikipedia: https://en.wikipedia.org/wiki/Suffix_array
* Basic algorithms and C++ code: https://web.stanford.edu/class/cs97si/suffix-array.pdf
* Some Python code: https://louisabraham.github.io/notebooks/suffix_arrays.html

#### **Problem:** finding substrings in long strings

* "Naive" method of finding length-$M$ substring in a length-$N$ string: $O(NM)$ operations. (**Exercise:** Why?) 
* Far too slow! E.g, human genome is a string of length $3\cdot 10^9$ in the alphabet $\{A,C,G,T\}$
* General idea of faster search: index substrings to avoid repeated comparisons

### Suffix tree

The tree containing all *suffixes* of the string:

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Suffix_tree_BANANA.svg/250px-Suffix_tree_BANANA.svg.png'>

In [1]:
A = 'banana$'
for k in range(len(A)):
    print ('Suffix %d:' %(k), A[k:])

# from IPython.display import Image
# Image(url= "Suffix_tree_BANANA.svg", width=200)

Suffix 0: banana$
Suffix 1: anana$
Suffix 2: nana$
Suffix 3: ana$
Suffix 4: na$
Suffix 5: a$
Suffix 6: $


If a suffix tree is available for A, finding a length-$M$ substring requires only $O(M)$ operations.

### Suffix arrays

A lightweight alternative to suffix trees.

**Definition.** Given a string A, the suffix array SA contains positions of suffixes of $A$ in lexicographic order: i.e., A[SA[k]:] is the $k$'th smallest suffix in A.

In [1]:
import numpy as np

A = 'ababa$'
N = len(A)
suffixL = [A[n:] for n in range(N)]
print(suffixL)
 
SA = np.argsort(suffixL) # the suffix array 
print ('SA:', SA)

print ('\nUnsorted suffixes:')
for n in range(N):
    print ('%d:' %(n), A[n:])

print ('\nSorted suffixes:')
for k in range(N):
    print ('%d:' %(SA[k]), A[SA[k]:])

['ababa$', 'baba$', 'aba$', 'ba$', 'a$', '$']
SA: [5 4 2 0 3 1]

Unsorted suffixes:
0: ababa$
1: baba$
2: aba$
3: ba$
4: a$
5: $

Sorted suffixes:
5: $
4: a$
2: aba$
0: ababa$
3: ba$
1: baba$


**Exercise:** With constructed SA, we can find a length-$M$ substring in a length-$N$ string using only $O(M\log N)$ operations:  repeated binary searches! 

### Typical problems efficiently solved using SA:
* Findind given substrings in long strings
* Finding the largest common substring in several strings
* Finding the number of different substrings in a string

### Construction of Suffix Array: the prefix-doubling algorithm

Idea: iterations; at step $m$ sort length-$2^m$ substrings 

* Start by sorting the single letters of A. 
* Sort length-2 substrings of A: represent them as pairs [A[k], A[k+1]], and sort such pairs lexicographically.  
* Continue by doubling the length of substrings: once length-$2^m$ substrings are sorted, sort length-$2^{m+1}$ substrings as pairs `[A[k:k+2**m], A[k+2**m:k+2**(m+1)]]` of length-$2^m$ substrings. 
* After at most $\log_2 N$ steps, all the suffixes are fully sorted. 

Complexity at each step is $O(N\log N)$, so the total complexity is $O(N\log^2 N)$

In [3]:
def invPerm(p):
    '''Invert the permutation p'''
    s = np.empty(p.size, p.dtype)
    s[p] = np.arange(p.size)
    return s


def getSA(A):
    if not type(A) is np.ndarray:
        A = np.array(list(A))
    N = len(A) 
    M = int(np.ceil(np.log2(N)))+1   # number of iterations
    
    # auxiliary arrays; row m stores results after m'th step:
    
    # positions of sorted length-(2**m) sequences in A
    P = np.zeros((M,N), dtype=int) 
    
    # rank (0, 1, etc.) of sorted length-(2**m) sequences after sorting
    Q = np.zeros((M,N), dtype=int)     
    
    # rank of sorted length-(2**m) sequences at its starting position in A;
    # padded by 0 on the right
    R = np.zeros((M,3*N), dtype=int) 

    for k in range(M):
        if k == 0:
            P[0] = np.argsort(A)
            Q[0][1:] = np.cumsum(A[P[0]][1:] != A[P[0]][:-1])
            R[0][:N] = Q[0][invPerm(P[0])]
        else:
            offset = 2**(k-1)
            r = np.lexsort((R[k-1, P[k-1]+offset], R[k-1, P[k-1]]))
            P[k] = P[k-1][r]
            # k'th rank increases iff (k-1)'th rank increases at least for one element of the pair    
            Q[k][1:] = np.cumsum(np.logical_or(R[k-1][P[k]][1:] != R[k-1][P[k]][:-1], 
                                          R[k-1][P[k]+offset][1:] != R[k-1][P[k]+offset][:-1]))
            R[k][:N] = Q[k][invPerm(P[k])]
            
            # early stopping if suffixes already fully sorted (max rank is N-1)
            if Q[k][-1] == N-1: 
                break
    
    SA = P[k]
    return SA, P[:k+1], Q[:k+1], R[:k+1]        

Check with the banana example:

In [4]:
A = 'banana$'
SA, P, Q, R = getSA(A)

print ('Suffix array:', SA)
print ()

print ('Sorted suffixes:')
for n in range(len(A)):
    print (A[SA[n]:])

print ()
print ('P:\n', P)
print ('Q:\n', Q)
print ('R:\n', R)

Suffix array: [6 5 3 1 0 4 2]

Sorted suffixes:
$
a$
ana$
anana$
banana$
na$
nana$

P:
 [[6 1 3 5 0 2 4]
 [6 5 1 3 0 2 4]
 [6 5 3 1 0 4 2]]
Q:
 [[0 1 1 1 2 3 3]
 [0 1 2 2 3 4 4]
 [0 1 2 3 4 5 6]]
R:
 [[2 1 3 1 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [3 2 4 2 4 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [4 3 6 2 5 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


### Test on SARRAY problem of Sphere Online Judge
http://www.spoj.com/problems/SARRAY/

Submit as a Python 2.7 or 3.7 program.

In [4]:
import sys
import numpy as np

def invPerm(p):
    'Invert the permutation p'
    s = np.empty(p.size, p.dtype)
    s[p] = np.arange(p.size)
    return s

def getSA(A):
    if not type(A) is np.ndarray:
        A = np.array(list(A))
    N = len(A) 
    M = int(np.ceil(np.log2(N)))+1   # number of iterations
    P = np.zeros((M,N), dtype=int) 
    Q = np.zeros((M,N), dtype=int)     
    R = np.zeros((M,3*N), dtype=int) 

    for k in range(M):
        if k == 0:
            P[0] = np.argsort(A)
            Q[0][1:] = np.cumsum(A[P[0]][1:] != A[P[0]][:-1])
            R[0][:N] = Q[0][invPerm(P[0])]
        else:
            offset = 2**(k-1)
            r = np.lexsort((R[k-1, P[k-1]+offset], R[k-1, P[k-1]]))
            P[k] = P[k-1][r]  
            Q[k][1:] = np.cumsum(np.logical_or(R[k-1][P[k]][1:] != R[k-1][P[k]][:-1], 
                                          R[k-1][P[k]+offset][1:] != R[k-1][P[k]+offset][:-1]))
            R[k][:N] = Q[k][invPerm(P[k])]

            if Q[k][-1] == N-1: 
                break

    SA = P[k]
    return SA, P[:k+1], Q[:k+1], R[:k+1]  

def main():
    A = sys.stdin.readline().rstrip('\n') + '$'
    SA = getSA(A)[0]
    for n in SA[1:]:
        print(n)

if __name__ == '__main__':
    main()  

### A C++ implementation for comparison:

```
#include <iostream>
#include <cstring>
#include <algorithm>
using namespace std;
 
// Structure to store information of a suffix
struct suffix
{
    int index; // To store original index
    int rank[2]; // To store ranks and next rank pair
};
 
// A comparison function used by sort() to compare two suffixes
// Compares two pairs, returns 1 if first pair is smaller
int cmp(struct suffix a, struct suffix b)
{
    return (a.rank[0] == b.rank[0])? (a.rank[1] < b.rank[1] ?1: 0):
               (a.rank[0] < b.rank[0] ?1: 0);
}
 
// This is the main function that takes a string 'txt' of size n as an
// argument, builds and return the suffix array for the given string
int *buildSuffixArray(char *txt, int n)
{
    // A structure to store suffixes and their indexes
    struct suffix suffixes[n];
 
    // Store suffixes and their indexes in an array of structures.
    // The structure is needed to sort the suffixes alphabatically
    // and maintain their old indexes while sorting
    for (int i = 0; i < n; i++)
    {
        suffixes[i].index = i;
        suffixes[i].rank[0] = txt[i] - '0';
        suffixes[i].rank[1] = ((i+1) < n)? (txt[i + 1] - '0'): -1;
    }
 
    // Sort the suffixes using the comparison function
    // defined above.
    sort(suffixes, suffixes+n, cmp);
 
    // At his point, all suffixes are sorted according to first
    // 2 characters.  Let us sort suffixes according to first 4
    // characters, then first 8 and so on
    int ind[n+1];  // This array is needed to get the index in suffixes[]
                 // from original index.  This mapping is needed to get
                 // next suffix.
    for (int k = 4; k < 2*n; k = k*2)
    {
        // Assigning rank and index values to first suffix
        int rank = 0;
        int prev_rank = suffixes[0].rank[0];
        suffixes[0].rank[0] = rank;
        ind[suffixes[0].index] = 0;
 
        // Assigning rank to suffixes
        for (int i = 1; i < n; i++)
        {
            // If first rank and next ranks are same as that of previous
            // suffix in array, assign the same new rank to this suffix
            if (suffixes[i].rank[0] == prev_rank &&
                    suffixes[i].rank[1] == suffixes[i-1].rank[1])
            {
                prev_rank = suffixes[i].rank[0];
                suffixes[i].rank[0] = rank;
            }
            else // Otherwise increment rank and assign
            {
                prev_rank = suffixes[i].rank[0];
                suffixes[i].rank[0] = ++rank;
            }
            ind[suffixes[i].index] = i;
        }
 
        // Assign next rank to every suffix
        for (int i = 0; i < n; i++)
        {
            int nextindex = suffixes[i].index + k/2;
            suffixes[i].rank[1] = (nextindex < n)?
                                  suffixes[ind[nextindex]].rank[0]: -1;
        }
 
        // Sort the suffixes according to first k characters
        sort(suffixes, suffixes+n, cmp);
    }
 
    // Store indexes of all sorted suffixes in the suffix array
    int *suffixArr = new int[n];
    for (int i = 0; i < n; i++)
        suffixArr[i] = suffixes[i].index;
 
    // Return the suffix array
    return  suffixArr;
}
 
// A utility function to print the suffix array
void printArr(int arr[], int n)
{
    for (int i = 0; i < n; i++)
        cout << arr[i] << endl;
}
 

int main()
{
    char txt[100000];
    std::cin.getline(txt,100000);
    int n = strlen(txt);
    int *suffixArr = buildSuffixArray(txt,  n);
    printArr(suffixArr, n);
    return 0;
}
```

### Finding substrings in a string
**Exercise:** Suffixes starting with a specific string form a *contiguous interval* in the suffix array (i.e. for any substring B in A there are $s,r$ such that the suffix $A[SA[n]:]$ starts with B iff $s \le n <r$). 

The boundaries of the interval can be found with $O(M\log N)$ operations by repeated dichotomy based on lexicographic comparison.

**Exercise:** The number of occurrences of substring B in A equals $r-s$.

In [5]:
def findSubstring(A, SA, B):
    '''
    Return the interval (s,r) such that for any s <= n <r the suffix A[SA[n]:] starts with B.
    '''
    N = len(A)
    N1 = len(B)
    l = 0; r = N
    while l < r:
        mid = (l+r) // 2
        if B > A[SA[mid]:SA[mid]+N1]:
            l = mid + 1
        else:
            r = mid
    s = l; r = N
    while l < r:
        mid = (l+r) // 2
        if B < A[SA[mid]:SA[mid]+N1]:
            r = mid
        else:
            l = mid + 1
    return (s, r)

#### Examples

In [3]:
A = 'banana$'
SA = getSA(A)[0]
print (SA)

for B in ['a', 'ana', 'aa']:
    print ('\nSubstring: ', B)
    (s,r) = findSubstring(A, SA, B)
    print ('The interval:', (s,r))
    print ('The number of occurrences:', r-s)
    for n in range(s,r):
        print (SA[n],': ', A[SA[n]:])

NameError: name 'getSA' is not defined

### The LCP array (Longest Common Substring)

LCP[n] = the length of the common prefix of suffixes A[SA[n]:] and A[SA[n+1]:] 

**Exercise:** For any $n < k$, the length of the common prefix of suffixes A[SA[n]:] and A[SA[k]:] equals $\min_{n\le s<k}$ LCP[s]. 

### Construction of the LCP array

Easy to construct from Suffix Array and the 2D array of ranks of sorted length-$2^m$ sequences.

Idea: check if suffixes have a common length-$2^M$ prefix, then check if they have a common prefix of length either $2^{M-1}$ or $(2^M+2^{M-1})$, etc.  

Then, construction of the whole LCP requires $O(N\log N)$ operations. 

In [6]:
def getLCP(SA, R):
    (M, N) = R.shape
    LCP = np.zeros((len(SA)-1,),dtype=int)
    for m in range(M-1)[::-1]:
        t = (R[m][SA[1:]+LCP] == R[m][SA[:-1]+LCP]).astype(int)
        LCP += (2**m)*t
    return LCP

Test: 

In [7]:
A = 'banana$'
SA, _, _, R = getSA(A)
LCP = getLCP(SA, R)
print("SA:", SA)
print ('LCP:', LCP)

for n in range(len(A)):
    print (SA[n], ': ', A[SA[n]:])

SA: [6 5 3 1 0 4 2]
LCP: [0 1 3 0 0 2]
6 :  $
5 :  a$
3 :  ana$
1 :  anana$
0 :  banana$
4 :  na$
2 :  nana$


### Finding the longest common substring

Let A anb B be two strings. How to find efficiently the longest common substring in them?

A possible approach:
* Concatenate A and B into a single string C (but insert some separator between A and B)
* Construct SA and LCP for C
* Find the maximum element in LCP among those lexicographically neighboring suffixes in C where one starts from A and another from B 

The whole procedure has complexity $O(N\log^2 N)$.

In [ ]:
def getLCS(A, B):
    N0 = len(A)
    N1 = len(B)
    C = A+'$'+B+' '
    SA, _, _, R = getSA(C)
    LCP = getLCP(SA, R)
    diff = (np.logical_xor(SA[:-1] < N0, SA[1:] < N0)).astype(int)
    n = np.argmax(LCP*diff)
    lcs = C[SA[n]:SA[n]+LCP[n]]
    return lcs

Test:

In [ ]:
A = 'applebananagrapefruitcucumberpotatograpefruit' 
B = 'peachorangeananastomatocherryorange'

print (getLCS(A, B))

anana


**Exercise:** solve problem http://www.spoj.com/problems/DISUBSTR/ (count distinct substrings in a string)  